# 01_data_inspection.ipynb

Objetivo: abrir un sample ROOT / NanoAOD, listar claves y trees, inspeccionar ramas relevantes (p. ej. muones) y generar histogramas básicos.

Requisitos previos: entorno con `uproot` y `awkward` instalados.

In [ ]:
# Imports
import os, glob
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

# Buscar sample ROOT en data/raw/ (primer match)
data_dir = "data/raw"
candidates = sorted(glob.glob(os.path.join(data_dir, "*.root")) + glob.glob(os.path.join(data_dir, "*.root.gz")))
if not candidates:
    print("No sample ROOT file found in data/raw/. Coloca un .root o crea un enlace a ~/Downloads/archivo.root")
else:
    sample = candidates[0]
    print("Sample ROOT file:", sample)

In [ ]:
if not candidates:
    raise SystemExit("Coloca un sample .root en data/raw/ y reejecuta la notebook.")
f = uproot.open(sample)
print("Top-level keys:")
pprint(f.keys())

In [ ]:
# Detectar TTrees u objetos Events
tree_names = []
for k in f.keys():
    try:
        obj = f[k]
        cname = obj.classname.decode() if isinstance(getattr(obj, 'classname', ''), (bytes, bytearray)) else str(getattr(obj, 'classname', ''))
        if "TTree" in cname:
            tree_names.append(k)
    except Exception:
        pass

# Fallback: muchos ROOTs usan "Events" como nombre
if not tree_names:
    if "Events" in f.keys():
        tree_names = ["Events"]

print("Detected TTrees:", tree_names)
if not tree_names:
    raise SystemExit("No TTrees found for inspección automática.")
tree_name = tree_names[0]
print("Using tree:", tree_name)
tree = f[tree_name]
print("Number of entries (approx):", getattr(tree, 'num_entries', 'unknown'))

In [ ]:
branches = list(tree.keys())
print("Number of branches:", len(branches))
print("First 60 branches:")
pprint(branches[:60])

In [ ]:
# Heurística: buscar ramas que contengan 'Muon' y 'pt' o 'mu_pt'
possible_pt_names = [b for b in branches if ("Muon" in b or "mu" in b.lower()) and "pt" in b.lower()]
print("Candidate pt branches:", possible_pt_names[:10])

pt_branch = None
for cand in ["Muon_pt", "Muon_pt_0", "muon_pt", "mu_pt", "Muon_pt0"]:
    if cand in branches:
        pt_branch = cand
        break
if not pt_branch and possible_pt_names:
    pt_branch = possible_pt_names[0]

if pt_branch:
    print("Using pt branch:", pt_branch)
    # Lectura segura de la rama pt (usa library="ak" para awkward)
    # entry_stop limitado para no usar demasiada memoria
    arr = tree[pt_branch].array(entry_stop=10000, library="ak")

    # Aplanar y convertir a numpy de forma segura
    try:
        arr_flat_ak = ak.flatten(arr)
        arr_flat = np.asarray(arr_flat_ak)
    except Exception:
        try:
            arr_flat = np.asarray(arr).ravel()
        except Exception:
            arr_flat = np.array([])

    if arr_flat.size > 0:
        print("Sample shape:", arr_flat.shape)
        print("min/max/mean:", float(np.nanmin(arr_flat)), float(np.nanmax(arr_flat)), float(np.nanmean(arr_flat)))
        plt.hist(arr_flat[:100000], bins=100, log=True)
        plt.xlabel("pt")
        plt.ylabel("counts (log)")
        plt.title(f"Histogram of {pt_branch}")
        plt.show()
    else:
        print("pt branch leída pero sin datos numéricos después de aplanar.")
else:
    print("No pt-like branch auto-detected. Busca manualmente en la lista de ramas.")

In [ ]:
# Ejemplo: intento de leer columnas típicas NanoAOD (ajusta segun tu archivo)
candidates = ["Muon_pt", "Muon_eta", "Muon_phi", "nMuon", "Muon_charge"]
found = [c for c in candidates if c in branches]
print("Found typical NanoAOD branches:", found)

for c in found:
    try:
        arr = tree[c].array(entry_stop=5000)
        print(c, "->", type(arr), "sample:", ak.to_list(arr[:5]))
    except Exception as e:
        print("Error reading", c, e)

Siguientes pasos sugeridos:

- Definir exactamente qué ramas y variables necesitas para el análisis (pT, eta, phi, vertex, MET).
- Usar `src/selection.py` para encapsular cortes y guardar un archivo reducido (parquet/npz) con solo variables necesarias.
- Crear notebooks posteriores para cálculo de ángulos, sincronización temporal y correlación con efemérides JPL.